In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d aryashah2k/indian-medicinal-leaves-dataset

indian-medicinal-leaves-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/indian-medicinal-leaves-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Avacado/5611.jpg  
  inflating: Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Avacado/5612.jpg  
  inflating: Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Avacado/5613.jpg  
  inflating: Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Avacado/5614.jpg  
  inflating: Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Avacado/5615.jpg  
  inflating: Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Avacado/5616.jpg  
  inflating: Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Avacado/5617.jpg  
  inflating: Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Avacado/5618.jpg  
  inflating: Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Avacado/5619.jpg  
  inflating: Indian Medicinal Leaves Image Datasets/Medicinal plant dataset/Avacado/5620.jpg  

In [ ]:
train_dir="/content/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset"

In [ ]:
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_data_leaves= tf.keras.preprocessing.image_dataset_from_directory(directory="/content/Indian Medicinal Leaves Image Datasets/Medicinal Leaf dataset",
                                                                            image_size=IMG_SIZE,
                                                                            label_mode="categorical",
                                                                            validation_split=0.2,
                                                                            subset="training",seed=42,
                                                                            batch_size=BATCH_SIZE)

Found 6900 files belonging to 80 classes.
Using 5520 files for training.


In [ ]:
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
aug=tf.keras.Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2)
])

In [ ]:
base=tf.keras.applications.EfficientNetB0(include_top=False)
base.trainable=True
for layer in base.layers[:-15]:
  layer.trainable=False
inputs=tf.keras.layers.Input((224,224,3))
x=aug(inputs)
x=base(x)
x=tf.keras.layers.GlobalAvgPool2D()(x)
outputs=tf.keras.layers.Dense(80,activation="softmax")(x)

In [ ]:
model1=tf.keras.Model(inputs,outputs)

In [ ]:
model1.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
model1.fit(train_data_leaves,epochs=5,validation_data=(train_data_leaves))

Epoch 1/5
173/173 [==============================] - 368s 2s/step - loss: 2.0010 - accuracy: 0.5509 - val_loss: 0.6549 - val_accuracy: 0.8154
Epoch 2/5
173/173 [==============================] - 324s 2s/step - loss: 0.6141 - accuracy: 0.8422 - val_loss: 0.4040 - val_accuracy: 0.8841
Epoch 3/5
173/173 [==============================] - 325s 2s/step - loss: 0.3547 - accuracy: 0.9125 - val_loss: 0.3114 - val_accuracy: 0.9096
Epoch 4/5
173/173 [==============================] - 326s 2s/step - loss: 0.2557 - accuracy: 0.9361 - val_loss: 0.2316 - val_accuracy: 0.9281
Epoch 5/5
173/173 [==============================] - 324s 2s/step - loss: 0.1977 - accuracy: 0.9484 - val_loss: 0.2409 - val_accuracy: 0.9207


In [ ]:
class_names=train_data_leaves.class_names

In [ ]:
import numpy as np
image = tf.keras.utils.load_img('/content/aloe-vera-herbs-of-zaytuna.jpg',target_size=(224,224))
input_arr = tf.keras.utils.img_to_array(image)
input_arr = np.array([input_arr])
preds=model1.predict(input_arr).argmax()
preds

1/1 [==============================] - 0s 27ms/step


0

In [ ]:
class_names[preds]

'Aloevera'

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16")

In [ ]:
from tensorflow.keras import layers
input_shape = (224, 224, 3)
base_model = tf.keras.applications.InceptionV3(include_top=False)
base_model.trainable = False
inputs = layers.Input(shape=input_shape, name="input_layer")
x = base_model(inputs, training=False) # set base_model to inference mode only
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
model2 = tf.keras.Model(inputs, outputs)
model2.compile(loss="categorical_crossentropy", # Use sparse_categorical_crossentropy when labels are *not* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:

model2.fit(train_data_leaves,epochs=15,validation_data=(train_data_leaves),callbacks=tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=0,patience=0,verbose=0,mode="auto",baseline=None,restore_best_weights=False,start_from_epoch=0))